###1. import

In [ ]:
!pip install konlpy
import pandas as pd # 데이터프레임 사용
from math import log # IDF 계산
from konlpy.tag import Kkma, Komoran, Okt # 형태소 분석
import itertools
from numpy import dot 
from numpy.linalg import norm

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 19.4 MB 1.2 MB/s 
     |████████████████████████████████| 453 kB 43.9 MB/s 


###2. Google 드라이브에 있는 .txt 파일 연결

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
f = open("/content/drive/MyDrive/정보검색/full_corpus.txt", 'r', encoding='cp949')
docs = f.read().split("<title>")
docs = [i.strip() for i in docs]
docs.pop(0)
f.close()

Mounted at /content/drive


###3. 함수 선언


In [ ]:
# 총 문서의 수
N = len(docs)

def tf(t, d):
  return d.count(t)

def idf(t):
  df = 0
  for doc in docs:
    df += t in doc
  return log(N/(df+1))

def tfidf(t, d):
  return tf(t,d)* idf(t)

def cos_sim(A, B):
  return dot(A, B)/(norm(A)*norm(B))

###4. Indexing time

In [ ]:
### 형태소 분류기 호출
komoran = Komoran()
okt = Okt()
apb=[] # 단어 저장(중복x)

### 추출된 단어 리스트에 다 때려박기
for i in docs:
    doc_i = komoran.nouns(i)
    apb.append(doc_i)
    doc_i = okt.nouns(i)
    apb.append(doc_i)
apb = list(set(itertools.chain(*apb))) # 중복단어 제거

### docs에 있는 단어 전부 tf-idf 계산
result=[]
for i in range(N):
    result.append([])
    d = docs[i]
    for j in range(len(apb)):
        t = apb[j]
        result[-1].append(tfidf(t,d))
result.append([]) # query를 docs처럼 취급
#df = pd.DataFrame(result, columns = apb)

###5. 검색하기 (Query time)

In [ ]:
result[N]=[0]*len(apb) # 검색시 항상 query 초기화
query = []
q = input("검색창 > ")
q_d = komoran.nouns(q)
query.append(q_d)
q_d = okt.nouns(q)
query.append(q_d)
query = list(set(itertools.chain(*query)))
#print(query)

### query에 있는 단어
for d in query:
    if d not in apb: 
        continue
    result[N][apb.index(d)] = tfidf(d,d)
#print(result[N])


answer = []
### 어느 doc과 유사한지 계산
for i in range(len(docs)):
    cos = cos_sim(result[i], result[N])
    if cos!=0:
        answer.append([i+1, cos])
        #print("doc",i+1,":", cos)
answer.sort(key=lambda x:-x[1])

#print(answer)

### 검색결과가 있을 때
print("검색결과는 >> doc ", end='')
cnt = 0
for i in range(len(answer)):
    if cnt==5: break
    print(answer[i][0], end=",")
    cnt+=1
print('\b')

검색창 > 대통령
검색결과는 >> doc 22,31,75,1,
